In [1]:
# Install LightGBM if not available (runs in notebook environment)
import sys
!{sys.executable} -m pip install --quiet lightgbm

In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from lightgbm import LGBMRegressor
import joblib
print('imports ready')

imports ready


In [3]:
# Load dataset
df = pd.read_csv('New Forest.txt')
print('loaded', df.shape)
display(df.head())
display(df.info())
display(df.isnull().sum())

loaded (1249, 18)


,State,Year,Total_Crop_Area_Ha,Forest_Cover_Area_SqKm,Very_Dense_Forest_SqKm,Mod_Dense_Forest_SqKm,Open_Forest_SqKm,Total_Forest_Recorded_SqKm,Forest_Percentage_Geographical,Scrub_Area_SqKm,NDVI_mean,RICE AREA (1000 ha),WHEAT AREA (1000 ha),SORGHUM AREA (1000 ha),MAIZE AREA (1000 ha),GROUNDNUT AREA (1000 ha),COTTON AREA (1000 ha),SUGARCANE AREA (1000 ha)
0,A & N Islands,1966,10730.65,6136.90,5678.0,684.0,381.0,6743.0,81.74,1.0,0.34,4292.26,2146.13,1073.07,1073.07,1073.07,536.53,536.53
1,A & N Islands,1967,6004.06,7155.20,5678.0,684.0,381.0,6743.0,81.74,1.0,0.29,2401.62,1200.81,600.41,600.41,600.41,300.20,300.20
2,A & N Islands,1968,5000.00,7170.92,5678.0,684.0,381.0,6743.0,81.74,1.0,0.44,2000.00,1000.00,500.00,500.00,500.00,250.00,250.00
3,A & N Islands,1969,12056.44,6675.65,5678.0,684.0,381.0,6743.0,81.74,1.0,0.32,4822.58,2411.29,1205.64,1205.64,1205.64,602.82,602.82
4,A & N Islands,1970,10067.57,6181.34,5678.0,684.0,381.0,6743.0,81.74,1.0,0.29,4027.03,2013.51,1006.76,1006.76,1006.76,503.38,503.38


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   State                           1249 non-null   object 
 1   Year                            1249 non-null   int64  
 2   Total_Crop_Area_Ha              1249 non-null   float64
 3   Forest_Cover_Area_SqKm          1249 non-null   float64
 4   Very_Dense_Forest_SqKm          1249 non-null   float64
 5   Mod_Dense_Forest_SqKm           1249 non-null   float64
 6   Open_Forest_SqKm                1249 non-null   float64
 7   Total_Forest_Recorded_SqKm      1249 non-null   float64
 8   Forest_Percentage_Geographical  1249 non-null   float64
 9   Scrub_Area_SqKm                 1249 non-null   float64
 10  NDVI_mean                       1249 non-null   float64
 11  RICE AREA (1000 ha)             1249 non-null   float64
 12  WHEAT AREA (1000 ha)            12

None

State                             0
Year                              0
Total_Crop_Area_Ha                0
Forest_Cover_Area_SqKm            0
Very_Dense_Forest_SqKm            0
Mod_Dense_Forest_SqKm             0
Open_Forest_SqKm                  0
Total_Forest_Recorded_SqKm        0
Forest_Percentage_Geographical    0
Scrub_Area_SqKm                   0
NDVI_mean                         0
RICE AREA (1000 ha)               0
WHEAT AREA (1000 ha)              0
SORGHUM AREA (1000 ha)            0
MAIZE AREA (1000 ha)              0
GROUNDNUT AREA (1000 ha)          0
COTTON AREA (1000 ha)             0
SUGARCANE AREA (1000 ha)          0
dtype: int64

In [4]:
# -----------------------------
# 1️⃣ Define ALL columns in dataset
# -----------------------------
ALL_COLS = [
    'Total_Crop_Area_Ha', 'Forest_Cover_Area_SqKm', 'Very_Dense_Forest_SqKm',
    'Mod_Dense_Forest_SqKm', 'Open_Forest_SqKm', 'Total_Forest_Recorded_SqKm',
    'Forest_Percentage_Geographical', 'Scrub_Area_SqKm', 'NDVI_mean',
    'RICE AREA (1000 ha)', 'WHEAT AREA (1000 ha)', 'SORGHUM AREA (1000 ha)',
    'MAIZE AREA (1000 ha)', 'GROUNDNUT AREA (1000 ha)', 'COTTON AREA (1000 ha)',
    'SUGARCANE AREA (1000 ha)'
]

missing = [c for c in ALL_COLS + ['State','Year'] if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}")
print("columns present")

# -----------------------------
# 2️⃣ Lag + Rolling features per State (still useful)
# -----------------------------
df_sorted = df.sort_values(['State','Year']).copy()

for col in ALL_COLS:
    df_sorted[f'{col}_lag1']  = df_sorted.groupby('State')[col].shift(1)
    df_sorted[f'{col}_roll3'] = (
        df_sorted.groupby('State')[col]
        .shift(1)
        .rolling(window=3, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )

lag_cols = [c for c in df_sorted.columns if c.endswith('_lag1')]
df_feat = df_sorted.dropna(subset=lag_cols + ['State','Year'] + ALL_COLS).reset_index(drop=True)
print("after lags/rolling:", df_feat.shape)

# -----------------------------
# 3️⃣ X = ALL COLUMNS (plus engineered ones)
# 4️⃣ y = ALL EXCEPT NDVI_mean
# -----------------------------
# y targets → everything except NDVI_mean
y_cols = [c for c in ALL_COLS if c != 'NDVI_mean']

y = df_feat[y_cols].copy()

# X → EVERYTHING (including NDVI_mean, lags, rolls, etc.)
X = df_feat.copy()

print("X shape:", X.shape)
print("y shape:", y.shape)

# -----------------------------
# 5️⃣ Train/test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("train/test:", X_train.shape, X_test.shape)

# -----------------------------
# 6️⃣ Target transforms
# -----------------------------
y_transformer = PowerTransformer(method='yeo-johnson')
y_train_trans = pd.DataFrame(
    y_transformer.fit_transform(y_train), columns=y_train.columns
)
y_test_trans = pd.DataFrame(
    y_transformer.transform(y_test), columns=y_test.columns
)

y_scaler = StandardScaler()
y_train_scaled = pd.DataFrame(
    y_scaler.fit_transform(y_train_trans), columns=y_train_trans.columns
)
y_test_scaled = pd.DataFrame(
    y_scaler.transform(y_test_trans), columns=y_test_trans.columns
)

print("y transform + scale complete")

# -----------------------------
# 7️⃣ Feature preprocessing
# -----------------------------
categorical = ['State']
numeric = [c for c in X.columns if c not in categorical]

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', ohe, categorical),
        ('num', 'passthrough', numeric)
    ],
    remainder='drop'
)

# -----------------------------
# 8️⃣ Model pipeline
# -----------------------------
base = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1
)

model = MultiOutputRegressor(base)

pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', model)
])

print("pipeline constructed")


columns present
after lags/rolling: (1213, 50)
X shape: (1213, 50)
y shape: (1213, 15)
train/test: (970, 50) (243, 50)
y transform + scale complete
pipeline constructed


In [7]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.utils.validation import check_is_fitted

# ---- Ensure pipeline is fitted ----
try:
    check_is_fitted(pipeline)
    print("Pipeline already fitted.")
except Exception:
    print("Fitting pipeline on training data...")
    pipeline.fit(X_train, y_train)
    print("Pipeline fitted.")

# ---- Predict ----
y_pred = pipeline.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=y_test.columns, index=y_test.index)

# ---- Metrics per target ----
results = []

for col in y_test.columns:
    r2 = r2_score(y_test[col], y_pred[col])
    # older scikit-learn versions don't accept `squared` → compute RMSE manually
    rmse = mean_squared_error(y_test[col], y_pred[col]) ** 0.5
    mae  = mean_absolute_error(y_test[col], y_pred[col])

    results.append({
        "Target": col,
        "R2": round(r2, 4),
        "RMSE": round(rmse, 3),
        "MAE": round(mae, 3)
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)

# ---- Overall averages ----
print("\nOverall:")
print("Avg R2 :", round(metrics_df['R2'].mean(), 4))
print("Avg RMSE:", round(metrics_df['RMSE'].mean(), 3))
print("Avg MAE :", round(metrics_df['MAE'].mean(), 3))


Pipeline already fitted.
                            Target      R2      RMSE      MAE
0               Total_Crop_Area_Ha  0.9980   907.405  241.356
1           Forest_Cover_Area_SqKm  0.9855  2493.992  656.213
2           Very_Dense_Forest_SqKm  0.9900   330.338   98.223
3            Mod_Dense_Forest_SqKm  0.9999    76.857   29.990
4                 Open_Forest_SqKm  0.9999    99.252   28.013
5       Total_Forest_Recorded_SqKm  0.9999   163.201   67.941
6   Forest_Percentage_Geographical  1.0000     0.198    0.084
7                  Scrub_Area_SqKm  1.0000    13.155    5.551
8              RICE AREA (1000 ha)  0.9981   359.935   94.345
9             WHEAT AREA (1000 ha)  0.9981   179.698   47.209
10          SORGHUM AREA (1000 ha)  0.9981    90.249   24.343
11            MAIZE AREA (1000 ha)  0.9984    83.847   21.224
12        GROUNDNUT AREA (1000 ha)  0.9981    89.955   23.230
13           COTTON AREA (1000 ha)  0.9999    16.770    6.687
14        SUGARCANE AREA (1000 ha)  0.9981   

In [8]:
import os
from sklearn.utils.validation import check_is_fitted

os.makedirs('models', exist_ok=True)

# ensure pipeline is fitted
try:
    check_is_fitted(pipeline)
except Exception:
    pipeline.fit(X_train, y_train)

# metadata / selected columns used for training
selected = {
    "feature_columns": X_train.columns.tolist(),
    "categorical": categorical,
    "numeric": numeric,
    "lag_cols": lag_cols,
    "y_columns": y_cols
}

# save artifacts
joblib.dump(pipeline, "models/pipeline.pkl")
joblib.dump(selected, "models/selected_columns.pkl")
joblib.dump(y_transformer, "models/y_transformer.pkl")
joblib.dump(y_scaler, "models/y_scaler.pkl")

print("Saved: ", sorted(os.listdir("models")))

Saved:  ['pipeline.pkl', 'selected_columns.pkl', 'y_scaler.pkl', 'y_transformer.pkl']
